In [2]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. 파생변수 생성
# ==========================================

def derive_features(df):

    age_order = {'만18-34세': 1, '만35-37세': 2, '만38-39세': 3, '만40-42세': 4, '만43-44세': 5, '만45-50세': 6, '알 수 없음': 0}
    df['나이_순서'] = df['시술 당시 나이'].map(age_order)
    df['나이x배아'] = df['나이_순서'] * df['이식된 배아 수']
    df['연령별 배아 효율'] = df['이식된 배아 수'] / (df['나이_순서'] + 1e-5) # '나이가 적으면서 배아가 많은 그룹' vs '나이가 많으면서 배아가 적은 그룹' 강조

    df['배아 발달 기간'] = df['배아 이식 경과일'] - df['난자 혼합 경과일']
    df['이식 비중'] = df['이식된 배아 수'] / (df['이식된 배아 수'] + df['저장된 배아 수'] + 1e-5)

    oocyte_cols = ['수집된 신선 난자 수', '혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '해동 난자 수']
    df['총 난자 수'] = df[oocyte_cols].fillna(0).sum(axis=1)
    df['배아 손실률'] = (df['총 난자 수'] - df['총 생성 배아 수']) / df['총 난자 수']
    
    df['배아 생성 효율'] = df['저장된 배아 수'] / (df['저장된 신선 난자 수'] + 1e-6)
    df['수정 효율'] = df['총 생성 배아 수'] / (df['혼합된 난자 수'] + 1e-6)
    df['선별 효율'] = df['저장된 배아 수'] / (df['총 생성 배아 수'] + 1e-6)
    
    def categorize_age_optimized(age_label):
        if age_label in ['만35-37세', '만38-39세']:
            return 1  # Middle
        elif age_label in ['만40-42세', '만43-44세', '만45-50세']:
            return 2  # Senior
        else:
            return 0  # '만18-34세'와 '알 수 없음'을 모두 0으로 처리
        
    df['나이_세분화'] = df['시술 당시 나이'].apply(categorize_age_optimized)

    def clean_treatment(text): 
        text = str(text).upper()
        if 'ICSI' in text:
            return 'ICSI'
        if 'IVF' in text:
            return 'IVF'
        if 'IUI' in text:
            return 'IUI'
        return 'Other'

    df['시술유형_정제'] = df['특정 시술 유형'].apply(clean_treatment)

    female_cols = ['불임 원인 - 여성 요인', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', 
                   '불임 원인 - 자궁내막증', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인']
    
    male_cols = ['불임 원인 - 남성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', 
                 '불임 원인 - 정자 면역학적 요인', '남성 주 불임 원인', '남성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인']
    
    df['여성_결함_점수'] = df[female_cols].sum(axis=1)
    df['남성_결함_점수'] = df[male_cols].sum(axis=1)
    
    drop_cols = ['시술 시기 코드', '특정 시술 유형', '시술 유형']
    df.drop(drop_cols + female_cols + male_cols, axis=1, inplace=True)
    
    return df

train_df = derive_features(train_df)
test_df = derive_features(test_df)

# 학습용 데이터 내에서 정답이 있는 '자체 테스트셋' 분리
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# ==========================================
# 3. 모델 학습 설정
# ==========================================

predictor = TabularPredictor(
    label=target, 
    eval_metric='roc_auc',
    path='ag_models_out',
).fit(
    train_data=train_data,
    time_limit=7200,
    presets='best_quality',
    included_model_types=['GBM', 'CAT', 'XGB', 'RF', 'XT', 'NN_TORCH'],
    num_stack_levels=1,
    num_bag_folds=5,
    refit_full=True
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]

# ==========================================
# 5. 리더보드 출력
# ==========================================

lb = predictor.leaderboard(val_data, silent=True)
display(lb.head())

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       4.12 GB / 16.00 GB (25.8%)
Disk Space Avail:   173.06 GB / 460.43 GB (37.6%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether 

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2_FULL,0.738729,NaN,roc_auc,2.705830,NaN,1156.430262,0.005955,NaN,22.631629,2,True,74
1,CatBoost_r70_BAG_L1,0.738726,0.739352,roc_auc,0.228771,0.213204,101.705258,0.228771,0.213204,101.705258,1,True,33
2,CatBoost_r9_BAG_L1,0.738664,0.739256,roc_auc,0.260637,0.249517,127.191046,0.260637,0.249517,127.191046,1,True,14
3,WeightedEnsemble_L2,0.738656,0.739939,roc_auc,12.850500,20.276842,3170.323006,0.004451,0.026124,22.631629,2,True,37
4,CatBoost_r69_BAG_L1,0.738631,0.739276,roc_auc,0.154437,0.208363,544.139547,0.154437,0.208363,544.139547,1,True,30


In [3]:
# --- 2. 피처 중요도 ---
fi = predictor.feature_importance(data=train_data.sample(n=min(5000, len(train_df)), random_state=42))
fi.to_excel('fi10.xlsx')

These features in provided data are not utilized by the predictor and will be ignored: ['ID']
Computing feature importance via permutation shuffling for 62 features using 5000 rows with 5 shuffle sets...
	579.19s	= Expected runtime (115.84s per shuffle set)
	373.51s	= Actual runtime (Completed 5 of 5 shuffle sets)


In [ ]:
# --- 3. 제출 파일 생성 ---
# submission = pd.read_csv('../Data/sample_submission.csv')
# submission['probability'] = final_probs.values

# # 현재 시간 가져오기 (예: 0206_1031)
# now = datetime.now().strftime('%m%d_%H%M')
# file_name = f"{now}_submission.csv"
# submission.to_csv(file_name, index=False)

# print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")